In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from random import seed, random
import matplotlib.pyplot as plt

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [18]:
df = pd.read_csv('./Magazine_Subscriptions.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'])

In [88]:
df.shape

(89689, 4)

In [156]:
# split to training and testing datasets by ratio of 6:4
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.3, random_state=42)

train_edges = []
test_edges = []

train_users = set()
test_users = set()

items = set()

for index, row in train_set.iterrows():
  items.add(row['asin'])
  train_edges.append((row['asin'], row['reviewerID']))
  train_users.add(row['reviewerID'])

for index, row in test_set.iterrows():
  items.add(row['asin'])
  test_edges.append((row['asin'], row['reviewerID']))
  test_users.add(row['reviewerID'])

In [165]:
# build graph with training sets
G = nx.Graph()
G.add_nodes_from(train_users, bipartite=0)
G.add_nodes_from(items, bipartite=1)
G.add_edges_from(train_edges)

In [166]:
# calculate the nodes (users') degree, and remove all edges for degree less or equal than 6
# the threshold means we only take account when a person subscribed more than 6 magazines
degree_set = dict(G.degree(train_users))
min_degree = 2
nodes_to_remove_set1 = [node for node, degree in degree_set.items() if min_degree > degree]
edges_to_remove_set1 = [(node, neighbor) for node in nodes_to_remove_set1 for neighbor in G.neighbors(node)]
G.remove_edges_from(edges_to_remove_set1)


In [167]:
# report the current edges
G.number_of_edges()

15224

In [168]:
# get neighbors
def get_neighbors(user_projection, user):

    neighbors = user_projection.neighbors(user)
    return neighbors

In [169]:
# get recommendation list based on information of neighbors
def recommendation(G, user_projection, user):
    recommendation_list = []
    neighbors = get_neighbors(user_projection, user)
    for neighbor in neighbors:
        for item in items:
            if(G.has_edge(neighbor, item)):
                if(not G.has_edge(user, item)):
                    recommendation_list.append(item)
                else:
                    continue
    return recommendation_list


#### Naive Model without any further analysis

In [170]:
# user-projection
user_projection_naive = nx.bipartite.weighted_projected_graph(G, train_users)

In [171]:
# clean network by removing all isolating nodes
user_projection_naive.remove_nodes_from(list(nx.isolates(user_projection_naive)))

In [172]:
# apply to test users to get the recommendation list
result_naive = {}
count_naive = 0
for user in test_users:
    if(user_projection_naive.has_node(user)):
        result_naive[user] = recommendation(G, user_projection_naive, user)
        count_naive += 1

In [173]:
# show one example of the recommendation
for key in result_naive:
    print(key, ':', result_naive[key])
    break

A34AI73ILO9E2O : ['B00005R8BA', 'B000063XKK', 'B001C4Q06G', 'B00005NIOW', 'B00EVV77A0', 'B00005N7QH', 'B00005N7OV', 'B00005NIOA', 'B00005N7OP', 'B00005N7QW', 'B00005NIOA', 'B000EGCIW8', 'B000FTJ7JQ', 'B00005N7QW', 'B00079RO7G', 'B000ICB4T6', 'B000K0YFVU', 'B000ILY9LW', 'B000XBBZ96', 'B00005N7OV', 'B000IOELH6', 'B00079RO7G', 'B00005UMOT', 'B000IJ84F6', 'B00EVV77A0', 'B00ZG7VQPU', 'B00005N7PN', 'B00007AZWJ', 'B00005NIOA', 'B000W3MB5M', 'B000UMJODW', 'B00015UYBO', 'B00005QJE0', 'B00ZG7VN8K', 'B00005N7QW', 'B000IOE9Y6', 'B000063XJL', 'B00XII1TZQ', 'B019ZV1FJO', 'B00XII1UD2', 'B004AAON6S', 'B00005N7RA', 'B00007IJZT', 'B01CF3ECNK', 'B001LF4EVO', 'B000FTJ7JQ', 'B00005N7PN', 'B000INCK4I', 'B01HBMU68U', 'B001LF4EVO', 'B01H6WOM9Y', 'B00005NIOO', 'B001LF4EVO', 'B000IOE9Y6', 'B000ILUYMK', 'B0037STB02', 'B0193CNAIY', 'B01844RZRE', 'B00005N7U1', 'B000IJ7RQ8', 'B00005N7Q1', 'B015GHGCSA', 'B000UMJODW', 'B00005NIOW', 'B015GHGCT4', 'B00007IJZT', 'B000IOEK7M', 'B00005N7Q1', 'B00005N7QG', 'B00005N7U1', 'B

In [174]:
# create recommendation edges for final test
result_edges_naive = []
for res in result_naive:
    for item in result_naive[res]:
        result_edges_naive.append((item, res))

In [175]:
# count how many recommended magazines are actually subscribed
count_res_naive = 0
for i in result_edges_naive:
    if i in test_edges:
        count_res_naive += 1
print(count_res_naive)

67694


In [176]:
# predict score
hitting_rate_naive = count_res_naive / len(result_edges_naive)
hitting_rate_naive

0.08904176257809931

In [177]:
import operator

#### consider betweenness_centrality to remove some edges

In [179]:
# find betweeness centrality for user projection and sort
ebet = nx.edge_betweenness_centrality(user_projection_naive)
sorted_ebet = sorted(ebet.items(), key=operator.itemgetter(1)) 
sorted_ebet[0:5]

[(('AC3FQLX5LLVY8', 'A35U1K8RDJAWFC'), 5.4505854555596596e-08),
 (('A12WZDAAVHBQ4U', 'A1HAO20JJGDOJ8'), 5.4505854555596596e-08),
 (('A12WZDAAVHBQ4U', 'A32GYTRTQRG8TI'), 5.4505854555596596e-08),
 (('A12WZDAAVHBQ4U', 'A33Q32BD2TTEF8'), 5.4505854555596596e-08),
 (('A12WZDAAVHBQ4U', 'ACT2AA008KB31'), 5.4505854555596596e-08)]

In [180]:
# remove node with small betweeness centrality
user_projection_bc = user_projection_naive.copy()

def remove_small_ebet(graph): 
    graph.remove_edge(*(sorted_ebet[0][0])) 
    return graph

for i in range(100):
    user_projection_bc=remove_small_ebet(user_projection_bc)
    sorted_ebet = sorted_ebet[1:]


In [181]:
# redo the recommendation process as naive method
result_bc = {}
count_bc = 0
for user in test_users:
    if(user_projection_bc.has_node(user)):
        result_bc[user] = recommendation(G, user_projection_bc, user)
        count_bc += 1

result_edges_bc = []
for res in result_bc:
    for item in result_bc[res]:
        result_edges_bc.append((item, res))

count_res_bc = 0
for i in result_edges_bc:
    if i in test_edges:
        count_res_bc += 1
print(count_res_bc)
hitting_rate_bc = count_res_bc / len(result_edges_bc)
hitting_rate_bc

67694


0.08904176257809931

Consider Jaccard centrality

In [182]:
def remove_edges_below_jaccard_threshold(graph, threshold):
    jaccard_coefficients = {}

    # Calculate Jaccard coefficients for all pairs of nodes
    for edge in graph.edges():
        node_i, node_j = edge

        neighbors_i = set(graph.neighbors(node_i))
        neighbors_j = set(graph.neighbors(node_j))

        # Calculate Jaccard coefficient
        jaccard_coefficient_value = len(neighbors_i.intersection(neighbors_j)) / len(neighbors_i.union(neighbors_j))

        # Store the Jaccard coefficient for the edge
        jaccard_coefficients[edge] = jaccard_coefficient_value

    # Identify edges with Jaccard coefficients below the threshold
    edges_to_remove = [edge for edge, coefficient in jaccard_coefficients.items() if coefficient < threshold]

    # Remove the identified edges from the graph
    graph.remove_edges_from(edges_to_remove)

    return graph

In [183]:
user_projection_jc = user_projection_naive.copy()


In [184]:
user_projection_jc_clean = remove_edges_below_jaccard_threshold(user_projection_jc, 0.5)


In [185]:
user_projection_jc_clean.number_of_edges()

299265

In [186]:
user_projection_jc_clean.remove_nodes_from(list(nx.isolates(user_projection_jc_clean)))
user_projection_jc_clean.number_of_nodes()

5530

In [187]:
result_jc = {}
count_jc = 0
for user in test_users:
    if(user_projection_jc_clean.has_node(user)):
        result_jc[user] = recommendation(G, user_projection_jc_clean, user)
        count_jc += 1

result_edges_jc = []
for res in result_jc:
    for item in result_jc[res]:
        result_edges_jc.append((item, res))

count_res_jc = 0
for i in result_edges_jc:
    if i in test_edges:
        count_res_jc += 1
print(count_res_jc)
hitting_rate_jc = count_res_jc / len(result_edges_jc)
hitting_rate_jc

41421


0.27682837990469633

In [188]:
data = {
    "Naive Model": [hitting_rate_naive],
    "Between Centrality Based": [hitting_rate_bc],
    "Jaccard Coefficient": [hitting_rate_jc]
}
final_report = pd.DataFrame(data, index = ["Accuracy"])
print(final_report)

          Naive Model  Between Centrality Based  Jaccard Coefficient
Accuracy     0.089042                  0.089042             0.276828


In [191]:
data = {
    "Naive Model": [hitting_rate_naive],
    "Between Centrality Based": [hitting_rate_bc],
    "Jaccard Coefficient": [hitting_rate_jc],
}
final_report = pd.DataFrame(data, index = ["Accuracy"])
print(final_report)

          Naive Model  Between Centrality Based  Jaccard Coefficient  \
Accuracy     0.089042                  0.089042             0.276828   

          Global Ranking Method  
Accuracy               0.125281  


In [194]:
# show one example of the recommendation
# show one example of the recommendation
temp = 0
for key in result_naive:
    print("one example of the naive method:")
    print(key, ':', result_bc[key])
    temp+=1
    if(temp >= 4):
        break
    
# for key in result_bc:
#     print("one example of the method including betweenness centrality:")
#     print(key, ':', result_bc[key])
#     break
# # show one example of the recommendation
# for key in result_jc:
#     print("one example of the method including jaccard similarity coefficient:")
#     print(key, ':', result_bc[key])
#     break

one example of the naive method:
A34AI73ILO9E2O : ['B00005R8BA', 'B000063XKK', 'B001C4Q06G', 'B00005NIOW', 'B00EVV77A0', 'B00005N7QH', 'B00005N7OV', 'B00005NIOA', 'B00005N7OP', 'B00005N7QW', 'B00005NIOA', 'B000EGCIW8', 'B000FTJ7JQ', 'B00005N7QW', 'B00079RO7G', 'B000ICB4T6', 'B000K0YFVU', 'B000ILY9LW', 'B000XBBZ96', 'B00005N7OV', 'B000IOELH6', 'B00079RO7G', 'B00005UMOT', 'B000IJ84F6', 'B00EVV77A0', 'B00ZG7VQPU', 'B00005N7PN', 'B00007AZWJ', 'B00005NIOA', 'B000W3MB5M', 'B000UMJODW', 'B00015UYBO', 'B00005QJE0', 'B00ZG7VN8K', 'B00005N7QW', 'B000IOE9Y6', 'B000063XJL', 'B00XII1TZQ', 'B019ZV1FJO', 'B00XII1UD2', 'B004AAON6S', 'B00005N7RA', 'B00007IJZT', 'B01CF3ECNK', 'B001LF4EVO', 'B000FTJ7JQ', 'B00005N7PN', 'B000INCK4I', 'B01HBMU68U', 'B001LF4EVO', 'B01H6WOM9Y', 'B00005NIOO', 'B001LF4EVO', 'B000IOE9Y6', 'B000ILUYMK', 'B0037STB02', 'B0193CNAIY', 'B01844RZRE', 'B00005N7U1', 'B000IJ7RQ8', 'B00005N7Q1', 'B015GHGCSA', 'B000UMJODW', 'B00005NIOW', 'B015GHGCT4', 'B00007IJZT', 'B000IOEK7M', 'B00005N7Q1